In [4]:
import os
import pandas as pd
import numpy as np
from video_frame import ffmpeg_cut
from bbox_keypoints import Databuilder
from kp_process import keypoints_fix


In [ ]:
video_dir = r"D:\pose\Eksik OFTler\OneDrive-2021-10-28\pain"
video_list = os.listdir(video_dir)
target_dir = r"D:\pose\pain\data\raw_frame"
ffmpeg_cut(video_dir, target_dir, video_list=video_list )

In [8]:
img_file = r"D:\pose\pain\data\raw_frame\binnur_front_trim"
#img_file_list = [os.path.join(img_file, item) for item in os.listdir(img_file)]

save_file = r"D:\pose\pain\data"
data_model = Databuilder(img_file, save_file, 600, 600, load_model=False)
data_model.pipeline(needcrop=False, pose=True)

In [2]:
import os
import pandas as pd
import numpy as np
import shutil

In [3]:
kp_dir = r"D:\pose\pain\data\annotation\fixed_kp"
crop_dir = r"D:\pose\pain\data\crop_img"
save_dir = r"D:\pose\pain\image_data\crop_img"
hdf_set = [os.path.join(kp_dir,item) for item in os.listdir(kp_dir) if item.endswith(".h5")]
column = [f'kp{item}' for item in range(34)]
feature_df = pd.DataFrame(columns=column)
for file in hdf_set:
    hdf_file = pd.read_hdf(file, "df_with_missing")
    hdf_file.columns = column
    feature_df = pd.concat([feature_df, hdf_file])

In [9]:
lab_dir = r"D:\pose\pain\image_data"
kp_set = set(feature_df.index)
actions = ["lie", "sit", "stand"]
x_list = []
label_list = []
for i, act in enumerate(actions):
    img_set = set(os.listdir(os.path.join(lab_dir, act)))
    img_list = list(img_set & kp_set)
    x_list.append(feature_df.loc[img_list])

    label_list.extend([i]*len(img_list))
x =  pd.concat(x_list).values
y = np.asarray(label_list)
    

In [24]:
len(x)

3299

In [21]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [22]:
Xtrain, Xtest,ytrain,ytest = train_test_split(x, y, test_size= 0.2,random_state=40)
svc = SVC(kernel='rbf',class_weight='balanced', decision_function_shape='ovo')
model = make_pipeline(StandardScaler(), svc)
model.fit(Xtrain, ytrain)
model.score(Xtest, ytest)

0.9651515151515152

In [23]:
yfit = model.predict(Xtest)
print(classification_report(ytest, yfit,target_names=["lie", "sit", "stand"]))

              precision    recall  f1-score   support

         lie       0.96      0.99      0.98       190
         sit       0.99      0.95      0.97       220
       stand       0.95      0.96      0.95       250

    accuracy                           0.97       660
   macro avg       0.97      0.97      0.97       660
weighted avg       0.97      0.97      0.97       660



In [20]:
yfit = svc.predict(Xtest)
print(classification_report(ytest, yfit,target_names=["lie", "sit", "stand"]))

              precision    recall  f1-score   support

         lie       0.96      0.98      0.97       190
         sit       0.99      0.93      0.96       220
       stand       0.93      0.96      0.94       250

    accuracy                           0.96       660
   macro avg       0.96      0.96      0.96       660
weighted avg       0.96      0.96      0.96       660



In [20]:

kp_dir = r"D:\pose\pain\data\annotation\fixed_kp"
crop_dir = r"D:\pose\pain\data\crop_img"
save_dir = r"D:\pose\pain\image_data\crop_img"
hdf_set = [os.path.join(kp_dir,item) for item in os.listdir(kp_dir) if item.endswith(".h5")]
column = [f'kp{item}' for item in range(34)]
feature_df = pd.DataFrame(columns=column)
for file in hdf_set:
    hdf_file = pd.read_hdf(file, "df_with_missing")

    selected_set = np.random.choice(hdf_file.index, 200, replace=False)
    video_path = os.path.join(crop_dir, "_".join(selected_set[0].split("_")[:-1]))
    select_df = hdf_file.loc[selected_set]
    select_df.columns = column
    feature_df = pd.concat([feature_df, select_df])
    for img in selected_set:
        shutil.copy(os.path.join(video_path, img), save_dir)


In [23]:
out_path = r"D:\pose\pain\image_data\annotation\kps"
feature_df.to_hdf(os.path.join(out_path, "kp_feature.h5"),"df_with_missing", format="table", mode="w")
feature_df.to_csv(os.path.join(out_path, "kp_feature.csv"))

In [51]:
import torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')
model.classes = [16]
model.multi_label = False

Using cache found in C:\Users\zhuhongyi/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2021-11-20 torch 1.10.0 CUDA:0 (GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients
Adding AutoShape... 


In [45]:
img = r"D:\pose\pain\data\raw_frame\cocker front\cocker front_0227.jpg"

In [55]:
model.iou = 0.45
model.conf = 0.02
model.classes = [16]

In [56]:

result = model(img)

In [57]:
result.show()

In [2]:
import pandas as pd
import numpy as np
import itertools

In [25]:
file = r"D:\pose\pain\data\annotation\fixed_kp\binnur_side_Trim.h5"
test_hdf = pd.read_hdf(file,"df_with_missing")


In [22]:
data_col = [f"kp{i}" for i in range(34)]
test_hdf.columns = data_col

In [23]:
test_hdf.head()

,kp0,kp1,kp2,kp3,kp4,kp5,kp6,kp7,kp8,kp9,...,kp24,kp25,kp26,kp27,kp28,kp29,kp30,kp31,kp32,kp33
binnur_side_Trim_0001.jpg,0.846607,0.084661,0.694218,0.135457,0.931267,-0.101593,-0.440236,0.237050,0.440236,-0.237050,...,-0.186254,0.660353,0.893277,0.603413,0.812743,0.541828,-0.321711,0.778878,-0.033864,0.829675
binnur_side_Trim_0002.jpg,0.922046,0.079033,0.763981,0.131721,1.009860,-0.096595,-0.430288,0.254660,0.430288,-0.254660,...,-0.149284,0.711293,0.705759,0.311978,0.886921,0.570791,-0.289786,0.834232,-0.008781,0.851795
binnur_side_Trim_0003.jpg,0.766652,0.127775,0.606933,0.159719,0.862483,-0.047916,-0.447214,0.223607,0.447214,-0.223607,...,-0.191663,0.654849,0.586606,0.287419,0.766652,0.527073,-0.319438,0.766652,-0.047916,0.782624
binnur_side_Trim_0004.jpg,0.798596,0.143747,0.638877,0.175691,0.862483,-0.047916,-0.447214,0.223607,0.447214,-0.223607,...,-0.191663,0.638877,0.586606,0.289678,0.766652,0.527073,-0.303466,0.766652,-0.047916,0.782624
binnur_side_Trim_0005.jpg,0.977802,0.052382,0.873038,0.192068,0.925420,-0.069843,-0.453980,0.209529,0.453980,-0.209529,...,-0.244451,0.646048,0.611127,0.296833,0.733352,0.523823,-0.349215,0.733352,-0.087304,0.768273


In [4]:
order_list = (pd.Series(test_hdf.index)).apply(
                lambda item: int(item.split("_")[-1].strip("0").split(".")[0]))

In [5]:
gap = np.append(order_list.values[1:], 0)
gap = gap - order_list.values
gap[-1] = -gap[-1]

In [6]:
end_point = np.arange(len(gap))[gap>9]
start_point = np.append([0], end_point[:-1])
ends = end_point[(end_point-start_point)>5] + 1
starts = start_point[(end_point-start_point)>5] + 1

In [54]:
end_point

array([ 47, 280, 287, 291, 308, 353, 408, 441, 492, 510, 512, 516, 549,
       578, 644, 645, 923])

In [39]:
order_list[645]

1265

In [33]:
crop_file = r"D:\pose\pain\data\annotation\crop(1)"
name = "binnur_side_Trim.h5"
img_inf_col = ["x", "y", "w", "h"]
data_col = [f"kp{i}" for i in range(34)]
def range_select(test_hdf,name, gap_len=9, clip_len=5):
        order_list = (pd.Series(test_hdf.index)).apply(
                lambda item: int(item.split("_")[-1].strip("0").split(".")[0]))
        gap = np.append(order_list.values[1:], 0)-order_list.values
        gap[-1] = -gap[-1]
        end_point = np.arange(len(gap))[gap>gap_len]
        start_point = np.append([0], end_point[:-1])
        ends = end_point[(end_point-start_point)>clip_len] + 1
        starts = start_point[(end_point-start_point)>clip_len] + 1
        starts[0] = 0 if starts[0] == 1 else starts[0]
        length = ends-starts
        
        clip_index = np.concatenate([list(range(s,e)) for s,e in zip(starts, ends)])
        index_list = test_hdf.index[clip_index]
        crop_inf = pd.read_hdf(os.path.join(crop_file, name))
        index = pd.Series(crop_inf.index).apply(lambda item:item.split("\\")[1])
        crop_inf.index = index
        crop_inf = (crop_inf.loc[index_list])[img_inf_col]
        
        clip_df = test_hdf.iloc[clip_index]
        clip_df.columns = data_col
        clip_df = pd.concat([clip_df, crop_inf], axis=1)
        end_new = list(itertools.accumulate(length))
        start_new = [0] + end_new[:-1]
        index = [name]*len(start_new)
        return pd.DataFrame({"starts":start_new, "ends":end_new, "length":length}, index=index), clip_df

In [34]:
label, data = range_select(test_hdf, name)

In [29]:
b = [list(range(s,e)) for s,e in zip(starts, ends)]

In [38]:
import csv
file = r"D:\pose\pain\data\annotation\split\train.csv"
container = []
with open(file) as f:
    reader = csv.reader(f)
    for line in reader:
        container.append(line)
del container[0]
for item in container:
    clip = [[item[0],i, i+5, item[-1]] for i in range(item[1],item[2]-5, 2)]



In [42]:
item = container[0]
clip = [[item[0],i, i+5, int(item[-1])] for i in range(int(item[1]),int(item[2])-5, 2)]

In [7]:
import pandas as pd
train = pd.read_hdf("D:/pose/pain/data/annotation/kp_valid/Akrep_OFT_Cam1_Trim.h5", "df_with_missing")

In [3]:
train.head()

,kp0,kp1,kp2,kp3,kp4,kp5,kp6,kp7,kp8,kp9,...,kp28,kp29,kp30,kp31,kp32,kp33,x,y,w,h
Akrep_OFT_Cam1_Trim_0007.jpg,4.853627,-0.832050,2.496151,-0.277350,4.576277,-3.328201,-0.416025,0.277350,0.416025,-0.277350,...,3.466876,9.707253,-3.189526,0.277350,-2.496151,3.050851,15.0,0.0,720.0,750.0
Akrep_OFT_Cam1_Trim_0008.jpg,1.919290,-1.111168,1.212183,-0.656599,1.565736,-1.767767,-0.353553,0.353553,0.353553,-0.353553,...,1.515229,3.131473,-0.202031,0.252538,-0.606092,0.858630,16.0,0.0,718.0,750.0
Akrep_OFT_Cam1_Trim_0015.jpg,0.010720,0.122620,1.046983,-0.066026,0.801744,-0.273536,-0.499911,-0.009432,0.499911,0.009432,...,0.386724,1.537462,-0.386724,0.707421,-0.386724,0.707421,8.0,0.0,734.0,750.0
Akrep_OFT_Cam1_Trim_0017.jpg,-0.071247,-1.104329,0.356235,-1.068706,0.000000,-1.246823,0.035624,0.498729,-0.035624,-0.498729,...,0.480154,2.213914,-0.890588,1.745553,-0.142494,1.353694,155.0,53.0,439.0,644.0
Akrep_OFT_Cam1_Trim_0020.jpg,-0.877896,-0.259378,-0.532652,-0.840383,-0.758183,-0.339187,0.438948,0.239426,-0.438948,-0.239426,...,-0.199522,0.518756,-0.199522,0.798087,0.498804,0.997609,191.0,200.0,367.0,302.0


In [5]:
import numpy as np

In [8]:
a = train.iloc[:10,:-4].values
b = np.stack([a]*5)
b.shape

(5, 10, 34)

In [7]:
a.shape

(34,)

In [6]:
a = train.iloc[:]
a = a.iloc[0]

In [11]:
(a[["w","h"]]<10).all()

False

In [51]:
train = train.applymap(lambda item:int(item))

In [58]:
train.iloc[0]

starts     0
ends      12
length    12
label      1
Name: binnur_front_trim.h5, dtype: int64

In [1]:
import torch
import time
logits = torch.tensor([[0.1, 0.3, 0.2, 0.4],
                       [0.5, 0.01, 0.9, 0.4]])
targets = torch.tensor([3, 0])

In [4]:
def acc(logits, targets,  k):
    
    
    values, indices = torch.topk(logits, k=k, sorted=True)
    y = torch.reshape(targets, [-1, 1])
    
    correct = (y == indices) * 1.  # 对比预测的K个值中是否包含有正确标签中的结果
    top_k_accuracy = torch.mean(correct) * k 
    #correct = ((y == indices).any(axis=1))*1.
    #top_k_accuracy = torch.mean(correct)
    return top_k_accuracy
    

In [5]:
acc(logits, targets, 2)

tensor(1.)

In [1]:
import torch.nn as nn

In [3]:
m = nn.Sigmoid()
loss = nn.BCELoss()
input = torch.randn(3, requires_grad=True)
target = torch.empty(3).random_(2)
output = loss(m(input), target)
#output.backward()

In [57]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
'''
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
output = loss(input, target)
output.backward()
'''

'\noutput.backward()\n# Example of target with class probabilities\ninput = torch.randn(3, 5, requires_grad=True)\ntarget = torch.randn(3, 5).softmax(dim=1)\noutput = loss(input, target)\noutput.backward()\n'

In [1]:
import time

In [6]:
print(time.perf_counter())
import tensorflow
print(time.perf_counter())
import time
print(time.perf_counter())

2660.9067509
2660.9068379
2660.9070053


In [48]:
acc(logits, targets, 1)

185023.9931925
185023.994045


In [27]:
torch.mean()

tensor(0.5000)

In [25]:
torch.mean((y == indices) * 1)

RuntimeError: mean(): input dtype should be either floating point or complex dtypes. Got Long instead.

In [25]:
import logging
file = r"..\out\train_record"

In [26]:
def setup_logging(out_dir):
    _FORMAT = "[%(levelname)s: %(filename)s: %(lineno)4d]: %(message)s"
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    logger.propagate = False
    plain_formatter = logging.Formatter(
        "[%(asctime)s][%(levelname)s] %(filename)s: %(lineno)3d: %(message)s",
        datefmt="%m/%d %H:%M:%S",
    )

    filename = os.path.join(out_dir, "stdout.log")
    fh = logging.StreamHandler(open(filename))
    fh.setLevel(logging.DEBUG)
    fh.setFormatter(plain_formatter)
    logger.addHandler(fh)

In [67]:
def setup_logging():
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    # set two handlers
    log_file = "{}.txt".format(__name__)
    # rm_file(log_file)
    fileHandler = logging.FileHandler(os.path.join(file, log_file), encoding='UTF-8', mode="w")
    fileHandler.setLevel(logging.DEBUG)
    consoleHandler = logging.StreamHandler()
    consoleHandler.setLevel(logging.DEBUG)

    # set formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    consoleHandler.setFormatter(formatter)
    fileHandler.setFormatter(formatter)

    # add
    logger.addHandler(fileHandler)
    logger.addHandler(consoleHandler)
    return logger
    #logger.info("test")


In [45]:
setup_logging()

<Logger __main__ (DEBUG)>

In [55]:

log = setup_logging()
log.info("gog")

[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
[2022-01-01 18:23:10] {739409473.py:2} INFO - gog
2022-01-01 18:23:10,666 - __main__ - INFO - gog
--- Logging error ---
Traceback (most recent call last):
  File "D:\Users\zhuhongyi\Anaconda3\envs\open-mmlab\lib\logging\__init__.py", line 1028, in emit
    stream.write(msg + self.terminator)
AttributeError: 'str' object has no attribute 'write'
Cal

In [68]:
def logger_config(log_path,logging_name):
    '''
    配置log
    :param log_path: 输出log路径
    :param logging_name: 记录中name，可随意
    :return:
    '''
    '''
    logger是日志对象，handler是流处理器，console是控制台输出（没有console也可以，将不会在控制台输出，会在日志文件中输出）
    '''
    # 获取logger对象,取名
    logger = logging.getLogger()
    # 输出DEBUG及以上级别的信息，针对所有输出的第一层过滤
    logger.setLevel(level=logging.DEBUG)
    # 获取文件日志句柄并设置日志级别，第二层过滤
    log_path = os.path.join(log_path, "test2.text")
    handler = logging.FileHandler(log_path, encoding='UTF-8')
    handler.setLevel(logging.INFO)
    # 生成并设置文件日志格式
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    # console相当于控制台输出，handler文件输出。获取流句柄并设置日志级别，第二层过滤
    console = logging.StreamHandler()
    console.setLevel(logging.DEBUG)
    # 为logger对象添加句柄
    logger.addHandler(handler)
    logger.addHandler(console)
    



In [71]:

#logger_config(log_path=file, logging_name="test")
logger = logging.getLogger("test")
logger.info("set2")

set2
set2
set2
set2
set2
--- Logging error ---
Traceback (most recent call last):
  File "D:\Users\zhuhongyi\Anaconda3\envs\open-mmlab\lib\logging\__init__.py", line 1028, in emit
    stream.write(msg + self.terminator)
AttributeError: 'str' object has no attribute 'write'
Call stack:
  File "D:\Users\zhuhongyi\Anaconda3\envs\open-mmlab\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "D:\Users\zhuhongyi\Anaconda3\envs\open-mmlab\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "D:\Users\zhuhongyi\Anaconda3\envs\open-mmlab\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "D:\Users\zhuhongyi\Anaconda3\envs\open-mmlab\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
    app.start()
  File "D:\Users\zhuhongyi\Anaconda3\envs\open-mmlab\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
    self.io_loop.start()
  File "D:\Users\zhuhongyi\Anacon

In [2]:
import numpy as np
npy_file = r"D:\pose\i3d_kinatics\kinetics-i3d\data\v_CricketShot_g04_c01_rgb.npy"

In [3]:
data = np.load(npy_file)

In [5]:
data.shape

(1, 79, 224, 224, 3)